<a href="https://colab.research.google.com/github/fresher96/arabic-speech-commands/blob/master/colab/arabic_commands_classical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Data Preparation

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

# Link: https://drive.google.com/file/d/1-LwzGIEsPr28kmt8TbSEn9vlQq4vosvQ/view?usp=sharing

GoogleDriveDownloader.download_file_from_google_drive(
                                    file_id='1-LwzGIEsPr28kmt8TbSEn9vlQq4vosvQ',
                                    dest_path='./ASC.rar')
!mkdir dataroot; cd dataroot; unrar x ../ASC.rar > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

data_root = './dataroot/'
complement = '_featuers.csv'
splits = ['train', 'val', 'test']

In [ ]:
df = {}
X = {}
y = {}
for split in splits:
    df[split] = pd.read_csv(data_root + split + complement)
    print(split, df[split].shape)
    X[split] = df[split].iloc[:, :-1]
    y[split] = df[split].iloc[:, -1]

train (7380, 1189)
val (2460, 1189)
test (2460, 1189)


In [ ]:
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV

X['train_val'] = X['train'].append(X['val'], ignore_index=True)
y['train_val'] = y['train'].append(y['val'], ignore_index=True)
split_index = [-1 if i < len(X['train']) else 0 for i in range(len(X['train_val']))]
pdsplit = PredefinedSplit(test_fold = split_index)

##### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()(weights='distance', algorithm='brute')
params = dict(
    n_neighbors=[6],
)

clf = RandomizedSearchCV(clf, params, cv=pdsplit, n_jobs=-1, verbose=1, n_iter=10, return_train_score=True)

clf.fit(X['train_val'], y['train_val'])

In [ ]:
clf.cv_results_

{'mean_fit_time': array([0.08313727]),
 'mean_score_time': array([1.56857467]),
 'mean_test_score': array([0.71504065]),
 'mean_train_score': array([1.]),
 'param_n_neighbors': masked_array(data=[6],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 6}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.71504065]),
 'split0_train_score': array([1.]),
 'std_fit_time': array([0.]),
 'std_score_time': array([0.]),
 'std_test_score': array([0.]),
 'std_train_score': array([0.])}

In [ ]:
print(clf.cv_results_['mean_train_score'], clf.cv_results_['mean_test_score'])
clf.score(X['test'], y['test'])

[1.] [0.71504065]


0.6772357723577236

##### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

params = dict(
    
)

clf = RandomizedSearchCV(clf, params, cv=pdsplit, n_jobs=-1, verbose=1, n_iter=10, return_train_score=True)

clf.fit(X['train_val'], y['train_val'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.9s finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   error_score=nan,
                   estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={}, pre_dispatch='2*n_jobs',
                   random_state=None, refit=True, return_train_score=True,
                   scoring=None, verbose=1)

In [ ]:
clf.cv_results_

{'mean_fit_time': array([0.14860249]),
 'mean_score_time': array([0.67531061]),
 'mean_test_score': array([0.74552846]),
 'mean_train_score': array([0.8597561]),
 'params': [{}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.74552846]),
 'split0_train_score': array([0.8597561]),
 'std_fit_time': array([0.]),
 'std_score_time': array([0.]),
 'std_test_score': array([0.]),
 'std_train_score': array([0.])}

In [ ]:
print(clf.cv_results_['mean_train_score'], clf.cv_results_['mean_test_score'])
clf.score(X['test'], y['test'])

[0.8597561] [0.74552846]


0.7117886178861789

##### Decesion Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

params = dict(
    max_depth=[650, 700, 750]
)

clf = RandomizedSearchCV(clf, params, cv=pdsplit, n_jobs=-1, verbose=2, n_iter=10, return_train_score=True)

clf.fit(X['train_val'], y['train_val'])

Fitting 1 folds for each of 3 candidates, totalling 3 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   52.9s finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
        

In [ ]:
clf.cv_results_

{'mean_fit_time': array([29.94455767, 29.94237924, 21.51351619]),
 'mean_score_time': array([0.01946044, 0.01955152, 0.01169038]),
 'mean_test_score': array([0.35731707, 0.36056911, 0.35650407]),
 'mean_train_score': array([1., 1., 1.]),
 'param_max_depth': masked_array(data=[650, 700, 750],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 650}, {'max_depth': 700}, {'max_depth': 750}],
 'rank_test_score': array([2, 1, 3], dtype=int32),
 'split0_test_score': array([0.35731707, 0.36056911, 0.35650407]),
 'split0_train_score': array([1., 1., 1.]),
 'std_fit_time': array([0., 0., 0.]),
 'std_score_time': array([0., 0., 0.]),
 'std_test_score': array([0., 0., 0.]),
 'std_train_score': array([0., 0., 0.])}

In [ ]:
print(clf.cv_results_['mean_train_score'], clf.cv_results_['mean_test_score'])
clf.score(X['test'], y['test'])

[1. 1. 1.] [0.35731707 0.36056911 0.35650407]


0.3589430894308943

##### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=700)

params = dict(
    n_estimators=[250]
)

clf = RandomizedSearchCV(clf, params, cv=pdsplit, n_jobs=-1, verbose=2, n_iter=10, return_train_score=True)

clf.fit(X['train_val'], y['train_val'])

Fitting 1 folds for each of 1 candidates, totalling 1 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.7min finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=700,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                         

In [ ]:
clf.cv_results_ #0.75894309 0.76869919

{'mean_fit_time': array([98.05575132]),
 'mean_score_time': array([0.37129831]),
 'mean_test_score': array([0.76138211]),
 'mean_train_score': array([1.]),
 'param_n_estimators': masked_array(data=[250],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 250}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.76138211]),
 'split0_train_score': array([1.]),
 'std_fit_time': array([0.]),
 'std_score_time': array([0.]),
 'std_test_score': array([0.]),
 'std_train_score': array([0.])}

In [ ]:
print(clf.cv_results_['mean_train_score'], clf.cv_results_['mean_test_score'])
clf.score(X['test'], y['test'])

[1.] [0.76138211]


0.7439024390243902

##### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1500)

params = dict(
    C=[1e-5, 1e-6]
)

clf = RandomizedSearchCV(clf, params, cv=pdsplit, n_jobs=-1, verbose=2, n_iter=10, return_train_score=True)

clf.fit(X['train_val'], y['train_val'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 1 folds for each of 2 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   32.5s finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=1500,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [1e-05, 1e-06]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=2)

In [ ]:
clf.cv_results_ # 0.74390244 0.96395664

{'mean_fit_time': array([32.28090525, 14.75960302]),
 'mean_score_time': array([0.02417493, 0.03655481]),
 'mean_test_score': array([0.74390244, 0.74390244]),
 'mean_train_score': array([0.96395664, 0.8802168 ]),
 'param_C': masked_array(data=[1e-05, 1e-06],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-05}, {'C': 1e-06}],
 'rank_test_score': array([1, 1], dtype=int32),
 'split0_test_score': array([0.74390244, 0.74390244]),
 'split0_train_score': array([0.96395664, 0.8802168 ]),
 'std_fit_time': array([0., 0.]),
 'std_score_time': array([0., 0.]),
 'std_test_score': array([0., 0.]),
 'std_train_score': array([0., 0.])}

In [ ]:
print(clf.cv_results_['mean_train_score'], clf.cv_results_['mean_test_score'])
clf.score(X['test'], y['test'])

[0.96395664 0.8802168 ] [0.74390244 0.74390244]


0.7577235772357723

# HMM

In [ ]:
!pip install hmmlearn

     |████████████████████████████████| 368kB 3.4MB/s 


In [ ]:
from hmmlearn import hmm
from collections import defaultdict

n_mfcc = 12
dataset = {}

for split in splits + ['train_val']:
    dataset[split] = defaultdict(list)

    num = 0
    for (index, row), label in zip(X[split].iterrows(), y[split]):
        feature = row.to_numpy()
        feature = np.reshape(feature, (-1, n_mfcc))
        # print(feature, feature.shape, label); input();
        dataset[split][label].append(feature)
        num += 1

    print(split, num)

train 7380
val 2460
test 2460
train_val 9840


In [ ]:
# adapted from: https://github.com/wblgers/hmm_speech_recognition_demo/blob/master/demo.py
from tqdm import tqdm

def train_GMMHMM(states_num=5, dataset=dataset['train']):
    GMMHMM_Models = {}

    idx = 1;
    for label in tqdm(dataset.keys()):
        print('itr %02d | training on %s'%(idx, label))
        idx += 1

        model = hmm.GaussianHMM(n_components=states_num, \
                                covariance_type='diag', n_iter=100)
        trainData = dataset[label]
        length = np.zeros([len(trainData), ], dtype=np.int)
        for m in range(len(trainData)):
            length[m] = trainData[m].shape[0]
        trainData = np.vstack(trainData)
        model.fit(trainData, lengths=length)  # get optimal parameters
        GMMHMM_Models[label] = model
    return GMMHMM_Models

hmmModels = train_GMMHMM()

  0%|          | 0/41 [00:00<?, ?it/s]

itr 01 | training on backward


  2%|▏         | 1/41 [00:13<08:50, 13.27s/it]

itr 02 | training on cancel


  5%|▍         | 2/41 [00:18<07:05, 10.90s/it]

itr 03 | training on close


  7%|▋         | 3/41 [00:26<06:16,  9.91s/it]

itr 04 | training on digit


In [ ]:
def test_GMMHMM(hmmModels=hmmModels, testDataSet=dataset['test']):
    score_cnt = 0
    score_tot = 0
    idx = 1
    for label in testDataSet.keys():
        score_cls = 0
        features = testDataSet[label]
        score_tot += len(features);
        for feature in features:
            scoreList = {}
            for model_label in hmmModels.keys():
                model = hmmModels[model_label]
                score = model.score(feature)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            # print("Test on true label ", label, ": predict result label is ", predict); input();
            if predict == label: score_cls += 1
        score_cnt += score_cls
        print("itr %02d | %s accuracy: %d/%d = %.2f"%(idx, label, score_cls, len(features), 100.0*score_cls/len(features)), "%")
        idx += 1
    print("accuracy: %d/%d = %.2f"%(score_cnt, score_tot, 100.0*score_cnt/score_tot), "%")


test_GMMHMM(hmmModels, dataset['test'])
# test_GMMHMM(hmmModels, dataset['val'])
# test_GMMHMM(hmmModels, dataset['train'])

itr 01 | backward accuracy: 1/60 = 1.67 %
itr 02 | cancel accuracy: 1/60 = 1.67 %
itr 03 | close accuracy: 1/60 = 1.67 %
itr 04 | digit accuracy: 1/60 = 1.67 %
itr 05 | direction accuracy: 4/60 = 6.67 %
itr 06 | disable accuracy: 21/60 = 35.00 %
itr 07 | down accuracy: 32/60 = 53.33 %
itr 08 | eight accuracy: 8/60 = 13.33 %
itr 09 | enable accuracy: 0/60 = 0.00 %
itr 10 | enter accuracy: 0/60 = 0.00 %
itr 11 | five accuracy: 24/60 = 40.00 %
itr 12 | forward accuracy: 8/60 = 13.33 %
itr 13 | four accuracy: 3/60 = 5.00 %
itr 14 | left accuracy: 7/60 = 11.67 %
itr 15 | move accuracy: 1/60 = 1.67 %
itr 16 | next accuracy: 0/60 = 0.00 %
itr 17 | nine accuracy: 11/60 = 18.33 %
itr 18 | no accuracy: 0/60 = 0.00 %
itr 19 | ok accuracy: 7/60 = 11.67 %
itr 20 | one accuracy: 26/60 = 43.33 %
itr 21 | open accuracy: 2/60 = 3.33 %
itr 22 | options accuracy: 1/60 = 1.67 %
itr 23 | previous accuracy: 6/60 = 10.00 %
itr 24 | receive accuracy: 0/60 = 0.00 %
itr 25 | record accuracy: 5/60 = 8.33 %
itr 2